<a href="https://colab.research.google.com/github/JMC110/Ungraded-Assignment-5/blob/master/TF_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

model Using Keras

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# instance of the model
model = MyModel()

Choosing Adam Optimizer

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Evaluation metrics

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
#test model
@tf.function
def test_step(images, labels):
 
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 0.007453539874404669, Accuracy: 99.76166534423828, Test Loss: 0.05777716636657715, Test Accuracy: 98.45999908447266
Epoch 2, Loss: 0.004956498742103577, Accuracy: 99.84666442871094, Test Loss: 0.06747692823410034, Test Accuracy: 98.43999481201172
Epoch 3, Loss: 0.004363030195236206, Accuracy: 99.85832977294922, Test Loss: 0.07960333675146103, Test Accuracy: 98.41999816894531
Epoch 4, Loss: 0.00345218856818974, Accuracy: 99.8933334350586, Test Loss: 0.06473801285028458, Test Accuracy: 98.54000091552734
Epoch 5, Loss: 0.003934214822947979, Accuracy: 99.87166595458984, Test Loss: 0.08338598906993866, Test Accuracy: 98.3699951171875
Epoch 6, Loss: 0.0025577431079000235, Accuracy: 99.91999816894531, Test Loss: 0.07825692743062973, Test Accuracy: 98.45999908447266
Epoch 7, Loss: 0.0018587016966193914, Accuracy: 99.94499969482422, Test Loss: 0.08274993300437927, Test Accuracy: 98.48999786376953
Epoch 8, Loss: 0.0025300506968051195, Accuracy: 99.92500305175781, Test Loss: 0.0895

**Test accuracy 98.55%**

In [0]:
ndarray = np.ones([3, 9])